In [1]:
%config Completer.use_jedi = False
%gui qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import patches as mpatches
from matplotlib import colors as mcol
from matplotlib.ticker import PercentFormatter
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from PIL import Image
from skimage import filters, feature, exposure, restoration
import skimage.morphology as skmorph
from skimage.segmentation import random_walker
from scipy import ndimage as ndi
import scipy as sp
import tifffile
import os
import glob
from tqdm.notebook import tqdm
import json
import pickle
import importlib
#from mayavi import mlab

In [3]:
from dnn_classifier import network_components as nc
from dnn_classifier import network_main as net_main
from dnn_classifier import dataset_handler as dat
from dnn_classifier import model

Is cuda available: True
CUDA version: 11.2
PyTorch version: 1.9.0
Cuda device name: GeForce RTX 3090


In [4]:
import torch
import torch.nn as nn

In [5]:
series_number = 11

working_dir = "/group/ag_cmb/mohsen_simtemp/data/silvio/machine_learning/"

output_dir = "./figures/series_" + str(series_number) + "/"

save_net_dir = "./saved_networks/"

expression_times = ["overnight", "24h", "48h", "72h"]
gains = ["500", "550", "650"]

index_of_data = 2
    
dataset_expression_time = expression_times[index_of_data]
dataset_scaling = ["fixed", "linear", "whitened", "channel_whitened"][2]
dataset_gain = gains[1]

ext_str = dataset_expression_time + "_" + dataset_scaling + "_gain_" + dataset_gain

save_net_file_name = save_net_dir + "confocal_channel_net_" + str(series_number) + "_trained_on_" + ext_str + ".pth"

dataset_file_name = working_dir + "new_input_mohsen_" + ext_str + ".pkl"

print(dataset_file_name)
print(save_net_file_name)

image_dir = "/group/ag_cmb/mohsen_simtemp/data/silvio/raw_lsm_data/"

with open(working_dir + 'image_database.json', 'r') as fp:
    image_database = json.load(fp)

/group/ag_cmb/mohsen_simtemp/data/silvio/machine_learning/new_input_mohsen_48h_whitened_gain_550.pkl
./saved_networks/confocal_channel_net_11_trained_on_48h_whitened_gain_550.pth


In [6]:
data_handler = dat.MyFluorophoreDataset(brightness_data_file_name=dataset_file_name,
                                        do_brightness_augmentation=True,
                                        brightness_augmentation_factor=0.3)

loading data from /group/ag_cmb/mohsen_simtemp/data/silvio/machine_learning/new_input_mohsen_48h_whitened_gain_550.pkl...
Brightness data loaded for: 
['ENDO', 'GFP', 'GalNact', 'KDEL', 'LifeAct', 'NLS', 'SNAP25', 'STX4', 'STX6', 'TOM70', 'Vti1a', 'Blank']
Data for ENDO
Brightness data size: (10, 10000)
Brightness data mean: 2.481795082034223
Brightness data std: 6.597398492798679
Data for GFP
Brightness data size: (10, 10000)
Brightness data mean: 0.34132340221452606
Brightness data std: 1.0836090659705717
Data for GalNact
Brightness data size: (10, 10000)
Brightness data mean: 3.2154718639077466
Brightness data std: 9.734387499187868
Data for KDEL
Brightness data size: (10, 10000)
Brightness data mean: 1.9702317508354983
Brightness data std: 5.884682936201093
Data for LifeAct
Brightness data size: (10, 10000)
Brightness data mean: 0.9815113405196683
Brightness data std: 2.7074464815733092
Data for NLS
Brightness data size: (10, 10000)
Brightness data mean: 2.016398166514262
Brightnes

In [7]:
save_figures = True

In [8]:
best_parameters = np.load(save_net_dir + 'best_params_alfa.npy', allow_pickle=True).item()

print(best_parameters)

{'width': 64, 'n_middle_layers': 3, 'resnet_stride': 3, 'cardinality': 4, 'batch_size': 458, 'lr': 0.00014860544347674695, 'use_resnet': True, 'use_dropout': False, 'dropout_rate': 0.2, 'use_bn': True}


In [9]:
net, train_loader, test_loader, optimizer = model.model_factory(data_handler, *model.separate_args(best_parameters))

In [10]:
net.load_state_dict(torch.load(save_net_file_name))

<All keys matched successfully>

In [11]:
avg_accuracy, class_accuracy, false_positive, false_negative = model.calc_accuracy(net, train_loader,
                                                                                  data_handler.n_proteins,
                                                                                  full_output=True)

print(f"Training accuracy = {avg_accuracy * 100.0} %")

Training accuracy = 97.01164879882917 %


In [12]:
def mpl_color (color_name):

    hex_color = mcol.get_named_colors_mapping()[color_name]
    h = hex_color.lstrip('#')
    
    rgbs = list(float(int(h[i:i + 2], 16)) / 255.0 for i in (0, 2 ,4))
    rgbs.append(1.0)
    
    return rgbs

print(data_handler.protein_names)

#protein_color = cm.tab20(np.linspace(0.0, 1.0, data_handler.n_proteins))

protein_color = dict()

protein_color["Vti1a"] = mpl_color('xkcd:magenta')
protein_color["GFP"] = mpl_color('xkcd:lime')
protein_color["TOM70"] = mpl_color('xkcd:lilac')#mpl_color('xkcd:slate blue')
protein_color["SNAP25"] = mpl_color('xkcd:burgundy')
#protein_color("Rab5a"] = mpl_color('xkcd:light orange')
protein_color["STX4"] = mpl_color('xkcd:dark orange')
protein_color["STX6"] = mpl_color('xkcd:light red')
protein_color["LifeAct"] = mpl_color('xkcd:dark sky blue')#mpl_color('xkcd:navy')
protein_color["NLS"] = mpl_color('xkcd:dark yellow')
protein_color["KDEL"] = mpl_color('xkcd:moss')
protein_color["GalNact"] = mpl_color('xkcd:beige')
protein_color["ENDO"] = mpl_color('xkcd:bright yellow')#mpl_color('xkcd:blue')
protein_color["Blank"] = np.array([0.0, 0.0, 0.0, 1.0])
edge_color = np.array([0.3, 0.3, 0.3, 1.0])

for protein_name in data_handler.protein_names:
    assert list(protein_color.keys()).index(protein_name) != -1

['ENDO', 'GFP', 'GalNact', 'KDEL', 'LifeAct', 'NLS', 'SNAP25', 'STX4', 'STX6', 'TOM70', 'Vti1a', 'Blank']


In [13]:
def scale_image_fixed(image):
    
    return image / 65535.0

def scale_image_linear(image):
    
    _max = np.amax(image)
    _min = np.amin(image)
    
    if _max > _min:
        return (image - _min) / (_max - _min)
    else:
        return image - _min

def scale_image_whitened(image):
    
    return (image - np.mean(image)) / np.std(image)

def scale_image_channel_whitened(image):
    
    _im = image.copy()
    
    channel_mean = np.mean(image, axis=1)
    channel_std = np.std(image, axis=1)
    
    assert _im.shape[0] == 10
    
    for i in range(_im.shape[0]):
        _im[i, :] = (_im[i, :] - channel_mean[i]) / channel_std[i]

    return _im

In [14]:
def get_cell_background(brightfield_image):
            
        brightfield_image = (brightfield_image - np.mean(brightfield_image)) / np.std(brightfield_image)

        bff = np.fft.fft2(brightfield_image)

        window = sp.signal.windows.gaussian(brightfield_image.shape[0], std=170.0)
        window = np.outer(window, window)

        bff *= window

        brightfield_image_enhanced = scale_image_linear(np.exp(-scale_image_linear(np.real(np.fft.ifft2(bff))) ** 2 / 0.2))

        seed = np.copy(brightfield_image_enhanced)
        seed[1:-1, 1:-1] = brightfield_image_enhanced.min()
        mask = brightfield_image_enhanced

        dilated = skmorph.reconstruction(seed, mask, method='dilation')
        cells = np.clip((scale_image_linear(brightfield_image_enhanced - dilated) - 0.03) * 200.0, 0.0, 1.0)

        cells = restoration.denoise_tv_bregman(cells, weight=0.4)

#             binary_mask = (cells > filters.threshold_otsu(cells)).astype(np.float32)

#             binary_mask = skmorph.erosion(binary_mask, selem2)
#             binary_mask = skmorph.closing(binary_mask, selem1)
        #binary_mask = skmorph.dilation(binary_mask, selem2)

#             markers = np.zeros(brightfield_image.shape, dtype=np.uint)
#             markers[brightfield_image < 0.01] = 1
#             markers[brightfield_image > 0.99] = 2

#             cell_edge = random_walker(brightfield_image, markers, beta=10, mode='bf')       

#             cell_edge = scale_image(filters.roberts(brightfield_image_enhanced).astype(np.float32))
#             cell_edge = cells#feature.canny(binary_mask).astype(np.float32)

        return cells

def crop_file_name(full_file_name):
    
    last_slash_ind = len(full_file_name) - 1

    while full_file_name[last_slash_ind] != "/":
        last_slash_ind -= 1
    
    cropped_name = full_file_name[last_slash_ind + 1:-4]
    
    return cropped_name

In [15]:
def predict_image_from_file(file_name, pred_threshold=0.0):
    
    ind_filter = [0, 1, 2, 3, 5, 6, 7, 8, 9, 10]

    cropped_name = crop_file_name(file_name)
    
    protein_name = "UNKNOWN"
    
    for _pr in data_handler.protein_names:
        if cropped_name.find(_pr) != -1:
            protein_name = _pr
    
#     if cropped_name.find("Blank") != -1:
#         protein_name = "MOCK"
    
    raw_image_from_file = tifffile.imread(file_name)
    
    if raw_image_from_file.ndim == 3:
        raw_image_from_file = raw_image_from_file.reshape((1, 1, *raw_image_from_file.shape))
    elif raw_image_from_file.ndim == 4:
        raw_image_from_file = raw_image_from_file.reshape((1, *raw_image_from_file.shape))
        
    raw_image_stack = raw_image_from_file[0, :, :, :, :].copy()
    
    brightfield_stack = raw_image_stack[:, 4, :, :].copy().astype(np.float32)
    raw_image_stack = raw_image_stack[:, ind_filter, :, :].copy().astype(np.float32)
    
    raw_image_brightness = np.clip(scale_image_linear(np.sum(np.abs(raw_image_stack), axis=1)), 0.0, 1.0)
    
    image_size = brightfield_stack.shape[1:]
   
    false_color_stack = []
    cell_edge_false_color_stack = []
    unprocessed_false_color_stack = []
    
    for image_slice, brightfield_slice, image_brightenss_slice in zip(raw_image_stack,
                                                                      brightfield_stack,
                                                                      raw_image_brightness):
        
        try:
            image_slice_scaled = globals()["scale_image_" + dataset_scaling](image_slice)
        except:        
            raise ValueError("Type of dataset scaling is unknown!")
            
        false_color_image = np.zeros((*image_size, 4))
        unprocessed_false_color_image = np.zeros((*image_size, 4))
        
        unprocessed_false_color_image[:, :, 3] = 1.0
        
        normalizing_layer = nn.Softmax(dim=-1)
        
        net.eval()

        with torch.no_grad():

            _im = image_slice_scaled.copy().reshape(dat.n_channels, -1).transpose().astype(np.float32)

            input_data = torch.from_numpy(_im).float().to(nc.cuda_0)
            raw_output = net(input_data)
            predicted = normalizing_layer(raw_output).cpu().numpy()
            
            entropy = np.mean(np.sum(-predicted * np.log(predicted + 1.0e-16), axis=1))

        correct_pick = 0.0
        wrong_pick = 0.0

        for i, data_protein_name in enumerate(data_handler.protein_names):

            # the intended protein in the single-transfect slice
            if data_protein_name == protein_name:
                correct_pick += np.sum(predicted[:, i])

                # make a simple false-color image based on the brightness of the original images when the protein-name is known
                for ch in range(3):
                    unprocessed_false_color_image[:, :, ch] = image_brightenss_slice[:, :] * protein_color[protein_name][ch]
            
            elif data_protein_name != 'Blank':
                wrong_pick += np.sum(predicted[:, i])
            
            predicted[predicted[:, i] < pred_threshold, i] = 0.0
            
            filtered_image = predicted[:, i].reshape(image_size)

            for ch in range(4):
                false_color_image[:, :, ch] += filtered_image[:, :] * protein_color[data_protein_name][ch]

#             false_color_image = scale_image_linear(false_color_image)
        
        # Getting the cell halos from the bright-field image
        cell_edge = scale_image_linear(get_cell_background(brightfield_slice))
        
        cell_edge_false_color_image = false_color_image.copy()
        
        for ch in range(4):
            cell_edge_false_color_image[:, :, ch] += cell_edge[:, :] * edge_color[ch]        
        
        false_color_image = np.clip(false_color_image, 0.0, 1.0)
        cell_edge_false_color_image = np.clip(cell_edge_false_color_image, 0.0, 1.0)

        false_color_stack.append(false_color_image.copy())
        cell_edge_false_color_stack.append(cell_edge_false_color_image.copy())
        unprocessed_false_color_stack.append(unprocessed_false_color_image.copy())
        
    false_color_stack = (np.array(false_color_stack) * 65535.0).astype(np.uint16)
    cell_edge_false_color_stack = (np.array(cell_edge_false_color_stack) * 65535.0).astype(np.uint16)
    unprocessed_false_color_stack = (np.array(unprocessed_false_color_stack) * 65535.0).astype(np.uint16)
    
    accuracy = correct_pick / (correct_pick + wrong_pick)
    
    return cropped_name, protein_name,\
            false_color_stack, cell_edge_false_color_stack, unprocessed_false_color_stack,\
            entropy, accuracy

In [16]:
def render_tomo(false_color_stack, save_figure_dir, cropped_name, pixel_size_z):
    
    assert false_color_stack.shape[1] == false_color_stack.shape[2]
    
    xx, yy = np.meshgrid(np.linspace(0.0, 1.0, false_color_stack.shape[1]),
                         np.linspace(0.0, 1.0, false_color_stack.shape[2]))

    fig = plt.figure(figsize=(10, 10))
    plt.tight_layout()

    ax = fig.add_subplot(projection='3d', frame_on=True)
    #fig.subplots_adjust(top=1.5, bottom=-0.5, wspace=0.0)
    fig.subplots_adjust(top=1.0, bottom=0.0, left=0.0, right=1.0, wspace=0.0, hspace=0.0)

    z = 0.0
    
    #pixel size in x and y direction is 300 nm
    dz = 1.0 / float(false_color_stack.shape[1]) * pixel_size_z / 300.0# * 0.5
    #dz = 0.03 * 17.0 / float(false_color_stack.shape[0])

    ax.set_facecolor('black')

    _box_color = 'white'

    _z = 0.0

    ax.plot3D([0.0, 0.0], [0.0, 1.0], [_z, _z], color=_box_color, zorder=-1)
    ax.plot3D([0.0, 1.0], [1.0, 1.0], [_z, _z], color=_box_color, zorder=-1)
    ax.plot3D([1.0, 1.0], [1.0, 0.0], [_z, _z], color=_box_color, zorder=-1)
    ax.plot3D([1.0, 0.0], [0.0, 0.0], [_z, _z], color=_box_color, zorder=-1)
    
    px = np.empty(0)
    py = np.empty(0)
    pz = np.empty(0)
    pc = np.empty((0, 4))
    
    for _zorder, _image in enumerate(tqdm(false_color_stack[:, :, :, :])):

        _color = _image.copy().astype(np.float64) / 65535.0

        valid = np.sum(_color[:, :, :3], axis=2) > 0.1

        zz = np.ones_like(xx) * z

        #print(float(zz[valid].shape[0]) / float(np.prod(zz.shape)))

        ax.plot3D([0.0, 0.0], [0.0, 0.0], [z, z + dz], color=_box_color, zorder=_zorder)
        ax.plot3D([0.0, 0.0], [1.0, 1.0], [z, z + dz], color=_box_color, zorder=_zorder)
        ax.plot3D([1.0, 1.0], [1.0, 1.0], [z, z + dz], color=_box_color, zorder=_zorder)
        ax.plot3D([1.0, 1.0], [0.0, 0.0], [z, z + dz], color=_box_color, zorder=_zorder)

        z += dz
        
        px = np.concatenate((px, xx[valid]))
        py = np.concatenate((py, yy[valid]))
        pz = np.concatenate((pz, zz[valid]))
        pc = np.concatenate((pc, _color[valid]), axis=0)

    ax.scatter3D(px, py, pz,
                 c=pc,
                 edgecolor='none',
                 marker='.', s=4.0, alpha=0.5,
                 depthshade=True, zorder=_zorder)

    _z = z
    
    #print(f"max z = {z * float(false_color_stack.shape[1])} pixels")
    
    ax.plot3D([0.0, 0.0], [0.0, 1.0], [_z, _z], color=_box_color, zorder=_zorder + 2)
    ax.plot3D([0.0, 1.0], [1.0, 1.0], [_z, _z], color=_box_color, zorder=_zorder + 2)
    ax.plot3D([1.0, 1.0], [1.0, 0.0], [_z, _z], color=_box_color, zorder=_zorder + 2)
    ax.plot3D([1.0, 0.0], [0.0, 0.0], [_z, _z], color=_box_color, zorder=_zorder + 2)


    ax.set_xlim(-0.01, 1.01)
    ax.set_ylim(-0.01, 1.01)
    ax.set_zlim(-0.51, 0.51)
    ax.set_axis_off()

    #scale = np.diag([1.0, 1.0, 0.1, 1.0])

    #def short_proj ():
    #    return np.dot(Axes3D.get_proj(ax), scale)

    #ax.get_proj = short_proj
    ax.view_init(20, 40)
    ax.dist = 9.0
    
    ax.set_rasterized(True)
    plt.margins(0.0, 0.0, 0.0)
    
    fig.savefig(save_figure_dir + "/tomo_" + cropped_name + '.pdf',
                    format = 'pdf', bbox_inches='tight', pad_inches = 0.0, transparent = False, dpi = 300)  
    plt.close ()

In [17]:
# def render_tomo_voxel (false_color_stack, save_figure_dir, cropped_name, pixel_size_z):
    

#     fig = mlab.figure(bgcolor=(0.0, 0.0, 0.0), fgcolor=None, engine=None, size=(800, 600))
    
#     mlab.pipeline.volume(mlab.pipeline.scalar_field(false_color_stack[:, :, :, 0]))
    
#     mlab.draw(fig)
    
#     return
    
#     def midpoints(x):
#         sl = ()
#         for i in range(x.ndim):
#             x = (x[sl + np.index_exp[:-1]] + x[sl + np.index_exp[1:]]) / 2.0
#             sl += np.index_exp[:]
#         return x

#     assert false_color_stack.shape[1] == false_color_stack.shape[2]
    
#     xx, yy, zz = np.meshgrid(np.linspace(0.0, 1.0, false_color_stack.shape[1] + 1),
#                              np.linspace(0.0, 1.0, false_color_stack.shape[2] + 1),
#                              np.linspace(0.0, false_color_stack.shape[0] / float(false_color_stack.shape[1]) * pixel_size_z / 300.0,
#                                          false_color_stack.shape[0] + 1))
    
#     valid = []
    
#     for _image in false_color_stack:

#         _color = _image.copy().astype(np.float64) / 65535.0

#         valid.append((np.sum(_color[:, :, :3], axis=2) > 0.1))
    
#     valid = np.array(valid, dtype=np.int32).transpose((1, 2, 0))

#     fig = plt.figure(figsize=(20, 20))
#     plt.tight_layout()

#     ax = fig.add_subplot(projection='3d', frame_on=True)
#     #fig.subplots_adjust(top=1.5, bottom=-0.5, wspace=0.0)
#     fig.subplots_adjust(top=1.0, bottom=0.0, left=0.0, right=1.0, wspace=0.0, hspace=0.0)

#     ax.voxels(xx, yy, zz, valid,
#           #facecolors=colors,
#           edgecolors=None,
#           linewidth=0.0)
    
#     return
#     z = 0.0
    
#     #pixel size in x and y direction is 300 nm
#     dz = 1.0 / float(false_color_stack.shape[1]) * pixel_size_z / 300.0# * 0.5
#     #dz = 0.03 * 17.0 / float(false_color_stack.shape[0])

#     ax.set_facecolor('black')

#     _box_color = 'white'

#     _z = 0.0

#     ax.plot3D([0.0, 0.0], [0.0, 1.0], [_z, _z], color=_box_color, zorder=-1)
#     ax.plot3D([0.0, 1.0], [1.0, 1.0], [_z, _z], color=_box_color, zorder=-1)
#     ax.plot3D([1.0, 1.0], [1.0, 0.0], [_z, _z], color=_box_color, zorder=-1)
#     ax.plot3D([1.0, 0.0], [0.0, 0.0], [_z, _z], color=_box_color, zorder=-1)

#     for _zorder, _image in enumerate(tqdm(false_color_stack[:, :, :, :])):

#         _color = _image.copy().astype(np.float64) / 65535.0

#         valid = np.sum(_color[:, :, :3], axis=2) > 0.1

#         zz = np.ones_like(xx) * z

#         #print(float(zz[valid].shape[0]) / float(np.prod(zz.shape)))

#         ax.plot3D([0.0, 0.0], [0.0, 0.0], [z, z + dz], color=_box_color, zorder=_zorder)
#         ax.plot3D([0.0, 0.0], [1.0, 1.0], [z, z + dz], color=_box_color, zorder=_zorder)
#         ax.plot3D([1.0, 1.0], [1.0, 1.0], [z, z + dz], color=_box_color, zorder=_zorder)
#         ax.plot3D([1.0, 1.0], [0.0, 0.0], [z, z + dz], color=_box_color, zorder=_zorder)

#         z += dz


#         ax.scatter3D(xx[valid], yy[valid], zz[valid],
#                      c=_color[valid],
#                      edgecolor='none',
#                      marker='.', s=5.0, alpha=0.5,
#                      depthshade=True, zorder=_zorder)

#     _z = z
    
#     print(f"max z = {z * float(false_color_stack.shape[1])} pixels")
    
#     ax.plot3D([0.0, 0.0], [0.0, 1.0], [_z, _z], color=_box_color, zorder=_zorder + 2)
#     ax.plot3D([0.0, 1.0], [1.0, 1.0], [_z, _z], color=_box_color, zorder=_zorder + 2)
#     ax.plot3D([1.0, 1.0], [1.0, 0.0], [_z, _z], color=_box_color, zorder=_zorder + 2)
#     ax.plot3D([1.0, 0.0], [0.0, 0.0], [_z, _z], color=_box_color, zorder=_zorder + 2)


#     ax.set_xlim(-0.01, 1.01)
#     ax.set_ylim(-0.01, 1.01)
#     ax.set_zlim(-0.51, 0.51)
#     ax.set_axis_off()

#     #scale = np.diag([1.0, 1.0, 0.1, 1.0])

#     #def short_proj ():
#     #    return np.dot(Axes3D.get_proj(ax), scale)

#     #ax.get_proj = short_proj
#     ax.view_init(20, 40)
#     ax.dist = 9.0
    
#     ax.set_rasterized(True)
#     plt.margins(0.0, 0.0, 0.0)
    

# #     fig.savefig(save_figure_dir + "/tomo_" + cropped_name + '.pdf',
# #                     format = 'pdf', bbox_inches='tight', pad_inches = 0.0, transparent = False, dpi = 300)  
# #     plt.close ()

## Processing the co-seeding samples

In [18]:
series = []

# series.append("001_lsm_files_overnight/20210415_Single_ON_NRXN_NLGN")
# series.append("002_lsm_files_24hrs/20210505_lsm_files_diff_gains/20210505_Single_NRXN_24hr_diff_gains")
# series.append("002_lsm_files_24hrs/20210129_Single_24hr_NRXN_NLGN")
# series.append("003_lsm_files_48hrs/20210325_Single_48hr_NRXN_NLGN")
# series.append("004_lsm_files_72hrs/001_single_transfect/zstacks/20210108_Single_72hr_NRXN_NLGN_Gain500")
# series.append("004_lsm_files_72hrs/001_single_transfect/zstacks/20201230_Single_72hr_NRXN_NLGN_Gain550")
# series.append("004_lsm_files_72hrs/004_coseeding/zstacks/20201221_CoSeeding_72hr_NRXN_NLGN")
# series.append("004_lsm_files_72hrs/004_coseeding/zstacks/20201221_CoSeeding_72hr_NRXN_NLGN_Gain550")
# series.append("004_lsm_files_72hrs/004_coseeding/zstacks/20201221_CoSeeding_72hr_NRXN_NLGN_zoom")
# series.append("004_lsm_files_72hrs/004_coseeding/zstacks/20201230_CoSeeding_72hr_NRXN_NLGN_Gain550")
# series.append("004_lsm_files_72hrs/004_coseeding/zstacks/20210108_CoSeeding_72hr_NRXN_NLGN_Gain500")
# series.append("006_Coseeding/20210616_Optimized_time/20210616_Optimized_time")
# series.append("006_Coseeding/20210709_all_nrxns_diff_nlgns/all_nb")
# series.append("006_Coseeding/20211205_48hrs")
series.append("006_Coseeding/20211209_Coseeding")

In [19]:
#mlab.init_notebook()

In [20]:
#name_template = "*ALLNRXNs_KDEL_NB_P1_CH11_z17_330nm.lsm"
name_template = "*.lsm"

for _series in series:

    sub_dir = image_dir + "2021_coseeding_series/" + _series + "/"
    
    save_figure_dir = output_dir + "/2021_coseeding/trained_on_" + dataset_expression_time + "/tomogram/" + _series
    os.makedirs(save_figure_dir, exist_ok=True)
    
    image_file_name = glob.glob(sub_dir + name_template)

    print(f"{len(image_file_name)} files found!")

    for _f in tqdm(image_file_name):

        cropped_name, protein_name, false_color_stack, cell_edge_false_color_stack, unprocessed_false_color_stack,\
            entropy, accuracy = predict_image_from_file(_f, pred_threshold=0.8)

        print(cropped_name, protein_name, entropy, accuracy)

        render_tomo(false_color_stack, save_figure_dir, cropped_name, 330.0)

36 files found!


  0%|          | 0/36 [00:00<?, ?it/s]

ALL_ALL_CS1_P3_CH11_z13_500nm UNKNOWN 0.019990282 0.0


  0%|          | 0/13 [00:00<?, ?it/s]

MOCK_CS2_P2_CH11_z13_500nm UNKNOWN 0.03199962 0.0


...

  0%|          | 0/13 [00:00<?, ?it/s]

2_7_8_10_CS1_P1_CH11_z13_500nm UNKNOWN 0.02359554 0.0


  0%|          | 0/13 [00:00<?, ?it/s]

ALL_ALL_CS1_P2_CH11_z13_500nm UNKNOWN 0.027974272 0.0


  0%|          | 0/13 [00:00<?, ?it/s]